# KNN

![knn_picture](http://i.imgur.com/gLBo1gX.png)

- 새로운 데이터가 들어왔을 때, 가장 가까운 k개의 이웃의 정보로 새로운 데이터의 분류(classification)를 예측
- __학습__의 개념이 없음, 새로운 데이터가 왔을 때 기존 데이터와의 거리를 계산할 뿐 → `Lazy Model`, `Instance Based Learning`이라고도 함

In [55]:
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifier().get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

위 파라미터들 중 `n_neighnors` , `metric`, `p` 를 살펴볼 것

## Hyper Parameter

### K의 값

- 탐색할 데이터의 수 
- `n_neighbors` 키워드 사용
- K의 값이 너무 작으면 데이터 포인트 하나하나를 따져 Overfitting 발생
- K의 값이 너무 크면 과하게 정규화되어 Underfitting 발생

![K크기조절](http://i.imgur.com/6Ub8CXe.png)

In [63]:
# 데이터로 확인하기
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

from sklearn.model_selection import cross_val_score
knn1 = KNeighborsClassifier(n_neighbors=1)
knn2 = KNeighborsClassifier(n_neighbors=5)
knn3 = KNeighborsClassifier(n_neighbors=10)
knn4 = KNeighborsClassifier(n_neighbors=20)
knn5 = KNeighborsClassifier(n_neighbors=50)
knns = [knn1, knn2, knn3, knn4, knn5]
for i in range(len(knns)):
    model = knns[i]
    scores = cross_val_score(model, X, y, cv=5)
    print(f'when K is {model.n_neighbors} : Cross Validation Mean: {round(scores.mean(), 4)} // Std: {round(scores.std(), 5)}')
    

when K is 1 : Cross Validation Mean: 0.96 // Std: 0.02494
when K is 5 : Cross Validation Mean: 0.9733 // Std: 0.02494
when K is 10 : Cross Validation Mean: 0.98 // Std: 0.02667
when K is 20 : Cross Validation Mean: 0.96 // Std: 0.03266
when K is 50 : Cross Validation Mean: 0.9133 // Std: 0.02667


- 지금은 데이터가 깔끔해서 큰 차이가 보이지는 않지만, 정확성이 높고 분산이 작은 5나 10이 좋아 보임

### Distance의 정의

- 어떤 거리 개념을 사용할 것인지
- `metric`, `p` 키워드를 통해 설정
-  기본값은 `metric='minkowski'`, `p=2`

> __참고 : 다양한 거리의 개념__
$$
\text{Manhattan Distance : } \sum_1^n{|x_i - y_i|} \\
\text{Euclidean Distance : } \sqrt{\sum_1^n{(x_i - y_i)^2}} \\
\text{일반화 => Minkowski Distance : } {\left( \sum_1^n{|x_i - y_i|^p} \right)}^\frac{1}{p} \\
$$

> __참고2 : 공분산 고려 거리__
$$
\text{Mahalanobis Distance : } \sqrt{(\vec{X} - \vec{Y})\Sigma^{-1}(\vec{X} - \vec{Y})} \\
\Sigma \text{ is covariance matrix} \\
$$
- Mahalanobis Distance 에서 p의 값을 파라미터로 사용할 수도 있음
- `metric` 인자를 __wminkovski__로 설정하여 가중치 사용할 수도 있음

In [73]:
# 데이터로 확인하기
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

from sklearn.model_selection import cross_val_score
knn1 = KNeighborsClassifier(p=3)
knn2 = KNeighborsClassifier(metric='euclidean')
knn3 = KNeighborsClassifier(metric='manhattan')


knns = [knn1, knn2, knn3]
for i in range(len(knns)):
    model = knns[i]
    scores = cross_val_score(model, X, y, cv=5)
    print(f'when metric is {model.metric} : Cross Validation Mean: {round(scores.mean(), 4)} // Std: {round(scores.std(), 5)}')

when metric is minkowski : Cross Validation Mean: 0.9733 // Std: 0.02494
when metric is euclidean : Cross Validation Mean: 0.9733 // Std: 0.02494
when metric is manhattan : Cross Validation Mean: 0.96 // Std: 0.03266


## 주의점 및 단점

### 주의점
1. 불균형 데이터에 대해 __사전 확률(prior probability)__ 고려 필요  
||정상|불량|
|---|---|---|
|사전확률|0.99|0.01|
|주변K개|0.8|0.2|

- 위와 같이 새로운 데이터의 주변 k개의 분포가 정상이 더 높아도, 분류는 불량으로 해야 하는 것이 맞는 상황 발생
- `weights` 인자에 함수를 할당하여 처리 가능

2. __정규화__ 필요  
||인구|위도|경도|
|---|---|---|---|
|서울|1000만|37|126|
|뉴욕|840만|43|75|

- 위와 같은 경우 거리를 계산할 때 인구의 영향이 너무 큼
- 따라서 모든 변수의 영향력이 비슷하도록 정규화 필요

### 단점
- 새로운 관측치와 기존의 관측치 간의 거리를 모두 계산해야 하므로 __계산 복잡도__가 높음
- __차원의 저주(Curse of Dimensionality)__

> 참조: [ratsgo's blog](https://ratsgo.github.io/machine%20learning/2017/04/17/KNN/), [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)